# 1. Load packages

In [1]:
from analysis_tools.common import *
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import sklearn
import cv2
import os

%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
sklearn.random.seed(RANDOM_STATE)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


# 2. Load dataset

In [2]:
train_full_data_meta = pd.read_csv(join(PATH.input, 'train_df.csv'), index_col=0)
test_data_meta       = pd.read_csv(join(PATH.input, 'test_df.csv'), index_col=0)

with ProgressBar():
    X_train_full = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.train)])
    X_test       = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.test)])

print("- Number of train full data:", len(X_train_full))
print("- Number of test data:", len(X_test))

[########################################] | 100% Completed |  7.8s
[########################################] | 100% Completed |  3.7s
- Number of train full data: 4277
- Number of test data: 2154


# 3. Training

In [3]:
IMG_SIZE    = 512
input_shape = (IMG_SIZE, IMG_SIZE, 3)

with ProgressBar():
    X_train_full = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_train_full]))
    X_test       = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_test]))

[########################################] | 100% Completed |  2.1s
[########################################] | 100% Completed |  0.9s


In [4]:
aug_model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

def preprocess(ds, training, batch_size, augment=True):
    ds = ds.cache().batch(batch_size)
    if training:
        ds = ds.shuffle(buffer_size=1000).prefetch(tf.data.AUTOTUNE)
        if augment:
            ds = ds.map(lambda X, y, sw: (aug_model(X), y, sw), num_parallel_calls=tf.data.AUTOTUNE)
    return ds

# fig, axes = plt.subplots(5, 15, figsize=(30, 10))
# for row, ax_cols in enumerate(axes):
#     for ax in ax_cols:
#         ax.imshow(aug_model(X_train_full[row]))
#         ax.axis('off')
# plt.show()

## 3.1 Classification(`class`)

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

y_train_full1 = train_full_data_meta[['class']]
y_enc1        = OneHotEncoder(sparse=False, dtype=bool)
y_train_full1 = y_enc1.fit_transform(y_train_full1)
n_classes1    = len(y_enc1.categories_[0])

X_train1, X_val1, y_train1, y_val1 = train_test_split(X_train_full, y_train_full1, stratify=y_train_full1)
sample_weight_train1 = compute_sample_weight(class_weight='balanced', y=y_train1.argmax(1))
sample_weight_val1   = compute_sample_weight(class_weight='balanced', y=y_val1.argmax(1))

train_ds1 = preprocess(tf.data.Dataset.from_tensor_slices((X_train1, y_train1, sample_weight_train1)), True, BATCH_SIZE)
val_ds1   = preprocess(tf.data.Dataset.from_tensor_slices((X_val1, y_val1, sample_weight_val1)), False, BATCH_SIZE)
test_ds1  = preprocess(tf.data.Dataset.from_tensor_slices(X_test), False, BATCH_SIZE)

print("- train1.shape:", X_train1.shape, y_train1.shape)
print("- val1.shape:", X_val1.shape, y_val1.shape)
print("- test.shape:", X_test.shape)

- train1.shape: (3207, 512, 512, 3) (3207, 15)
- val1.shape: (1070, 512, 512, 3) (1070, 15)
- test.shape: (2154, 512, 512, 3)


In [6]:
from tensorflow_addons.metrics import F1Score

def build_model(n_classes, strategy):
    def build_fine_tuning_model(model, base_model):
        with strategy.scope():
            base_model.trainable = True
            model.compile(optimizer=keras.optimizers.Adam(2e-4), loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
        return model, base_model
        
    with strategy.scope():
        base_model = keras.applications.EfficientNetB0(include_top=False, input_shape=input_shape)
        base_model.trainable = False

        inputs  = keras.Input(input_shape)
        hidden  = base_model(inputs, training=False)
        hidden  = keras.layers.GlobalAveragePooling2D()(hidden)
        outputs = keras.layers.Dense(n_classes, activation='softmax')(hidden)
        model   = keras.Model(inputs, outputs)
        
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
    
    return model, base_model, build_fine_tuning_model

In [7]:
from analysis_tools.modeling import *

model1, base_model1, build_fine_tuning_model1 = build_model(n_classes1, strategy)
model1.fit(train_ds1, validation_data=val_ds1, epochs=5, callbacks=get_callbacks(patience=5, plot_path=join(PATH.result, 'proposed4', 'classification_class')))

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

## 3.2 Classification(`label`)

In [8]:
train_full_data_meta.head()

,file_name,class,state,label
index,,,,
0,10000.png,transistor,good,transistor-good
1,10001.png,capsule,good,capsule-good
2,10002.png,transistor,good,transistor-good
3,10003.png,wood,good,wood-good
4,10004.png,bottle,good,bottle-good


In [9]:
X_train2, X_val2, y_train2, y_val2 = {}, {}, {}, {}
y_enc2             = {}
n_classes2         = {}
train_ds2, val_ds2 = {}, {}
for c in tqdm(train_full_data_meta['class'].unique()):
    idxs             = train_full_data_meta.query(f"`class` == '{c}'").index
    X_train_full2    = X_train_full[idxs]

    y_enc2[c]        = OneHotEncoder(sparse=False, dtype=bool)
    y_train_full2    = y_enc2[c].fit_transform(train_full_data_meta[['label']].loc[idxs])
    n_classes2[c]    = len(y_enc2[c].categories_[0])
    
    X_train2[c], X_val2[c], y_train2[c], y_val2[c] = train_test_split(X_train_full2, y_train_full2, stratify=y_train_full2)
    sample_weight_train2 = compute_sample_weight(class_weight='balanced', y=y_train2[c].argmax(1))
    sample_weight_val2   = compute_sample_weight(class_weight='balanced', y=y_val2[c].argmax(1))

    train_ds2[c] = preprocess(tf.data.Dataset.from_tensor_slices((X_train2[c], y_train2[c], sample_weight_train2)), True, BATCH_SIZE)
    val_ds2[c]   = preprocess(tf.data.Dataset.from_tensor_slices((X_val2[c], y_val2[c], sample_weight_val2)), False, BATCH_SIZE)

100%|██████████| 15/15 [00:12<00:00,  1.22it/s]


In [56]:
model2 = {}
for c in tqdm(train_full_data_meta['class'].unique()):
    if c != 'zipper':
        continue
        
    model2[c], base_model2, build_fine_tuning_model2 = build_model(n_classes2[c], strategy)
    model2[c].fit(train_ds2[c], validation_data=val_ds2[c], epochs=1000, callbacks=get_callbacks(patience=10, plot_path=join(PATH.result, 'proposed4', f'classification_label_{c}')))
    if c != 'zipper':  # 'zipper': fine tuning is not good (why?)
        model2[c], base_model2 = build_fine_tuning_model2(model2[c], base_model2)
        model2[c].fit(train_ds2[c], validation_data=val_ds2[c], epochs=1000, callbacks=get_callbacks(patience=10, plot_path=join(PATH.result, 'proposed4', f'classification_label_fine_tuning_{c}')))

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1/1000
8/8 [==============================] - 25s 1s/step - loss: 2.2240 - f1_score: 0.0664 - val_loss: 2.1773 - val_f1_score: 0.1113
Epoch 2/1000
8/8 [==============================] - 2s 91ms/step - loss: 2.1361 - f1_score: 0.1374 - val_loss: 2.1244 - val_f1_score: 0.1638
Epoch 3/1000
8/8 [==============================] - 2s 90ms/step - loss: 2.0616 - f1_score: 0.0422 - val_loss: 2.0781 - val_f1_score: 0.1124
Epoch 4/1000
8/8 [==============================] - 2s 86ms/step - loss: 2.0586 - f1_score: 0.0973 - val_loss: 2.0488 - val_f1_score: 0.3071
Epoch 5/1000
8/8 [==============================] - 2s 91ms/step - loss: 2.1065 - f1_score: 0.1380 - val_loss: 2.0332 - val_f1_score: 0.2380
Epoch 6/1000
8/8 [==============================] - 2s 88ms/step - loss: 1.9582 - f1_score: 0.2631 - val_loss: 2.0024 - val_f1_score: 0.2142
Epoch 7/1000
8/8 [==============================] - 2s 92ms/step - loss: 1.9663 - f1_score: 0.1579 - val_loss: 2.0059 - val_f1_score: 0.0355
Epoch 8/1000
8

Epoch 56/1000
8/8 [==============================] - 2s 239ms/step - loss: 1.3287 - f1_score: 0.5910 - val_loss: 1.6062 - val_f1_score: 0.3786
Epoch 57/1000
8/8 [==============================] - 2s 87ms/step - loss: 1.3471 - f1_score: 0.5379 - val_loss: 1.6238 - val_f1_score: 0.2786
Epoch 58/1000
8/8 [==============================] - 2s 88ms/step - loss: 1.4198 - f1_score: 0.4392 - val_loss: 1.6079 - val_f1_score: 0.3718
Epoch 59/1000
8/8 [==============================] - 2s 91ms/step - loss: 1.4267 - f1_score: 0.4074 - val_loss: 1.5932 - val_f1_score: 0.4552
Epoch 60/1000
8/8 [==============================] - 2s 91ms/step - loss: 1.3811 - f1_score: 0.5047 - val_loss: 1.5735 - val_f1_score: 0.4570
Epoch 61/1000
8/8 [==============================] - 2s 90ms/step - loss: 1.4043 - f1_score: 0.6184 - val_loss: 1.5702 - val_f1_score: 0.4991
Epoch 62/1000
8/8 [==============================] - 2s 88ms/step - loss: 1.3642 - f1_score: 0.4967 - val_loss: 1.5784 - val_f1_score: 0.3822
Epoch

100%|██████████| 15/15 [03:30<00:00, 14.06s/it]

Epoch 00071: early stopping


# 4. Evaluation

In [57]:
submission           = pd.read_csv(join(PATH.input, 'sample_submission.csv'), index_col=0)
submission_file_path = join(PATH.output, 'proposed4.csv')

pred_test1_oh  = model1.predict(test_ds1)                 # class(int)
pred_test1_str = y_enc1.inverse_transform(pred_test1_oh)  # class(str)
for c in tqdm(np.unique(pred_test1_str)):
    idxs = np.where(pred_test1_str == c)[0]
    X_c  = X_test[idxs]
    pred_test2_oh                 = model2[c].predict(X_c)
    submission.loc[idxs, 'label'] = y_enc2[c].inverse_transform(pred_test2_oh).flatten()

submission.to_csv(submission_file_path)
submission

100%|██████████| 15/15 [00:17<00:00,  1.16s/it]


,label
index,
0,tile-glue_strip
1,grid-good
2,transistor-good
3,tile-gray_stroke
4,tile-good
...,...
2149,tile-gray_stroke
2150,screw-good
2151,grid-good


In [ ]:
as

# 5. Submission

In [58]:
from dacon_submit_api.dacon_submit_api import post_submission_file

result = post_submission_file(
    submission_file_path,
    '137ff236e305f302819b930b3b5b72e948603f23c5249a516c32b536d5187a03', 
    '235894',
    '어스름한 금요일 밤에',
    get_name(submission_file_path)
)

{'isSubmitted': True, 'detail': 'Success'}
